In [30]:
import pandas as pd
from src.eventlog_utils import convert_log
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from log_distance_measures.config import EventLogIDs, AbsoluteTimestampType, discretize_to_hour
from log_distance_measures.control_flow_log_distance import control_flow_log_distance
from log_distance_measures.n_gram_distribution import n_gram_distribution_distance
from log_distance_measures.absolute_event_distribution import absolute_event_distribution_distance
from log_distance_measures.case_arrival_distribution import case_arrival_distribution_distance
from log_distance_measures.circadian_event_distribution import circadian_event_distribution_distance
from log_distance_measures.relative_event_distribution import relative_event_distribution_distance
from log_distance_measures.work_in_progress import work_in_progress_distance
from log_distance_measures.cycle_time_distribution import cycle_time_distribution_distance
from src.res_based_ced import resource_based_circadian_event_distribution_distance

In [31]:
case_studies = {
    1: 'Consulta',
    2: 'Production',
    3: 'Purchasing',
    4: 'bpi12',
    5: 'bpi17'
}

In [32]:
# chose case study
case_study = case_studies[3]

# Choose if our approach or Sota
our_approach = True
Sota = not(our_approach)
approach = 'DSIM' if Sota else None
print(f'Case study: {case_study}')

Case study: Production


In [33]:
log_real = xes_importer.apply(f'data/{case_study}/logTest.xes')
log_real = pm4py.convert_to_dataframe(log_real) 

if our_approach:
    log_sim = pd.read_csv(f'simulations/{case_study}/sim.csv')
    print('imported log from our approach')

elif Sota:
    log_sim = pd.read_csv(f'Sota/{case_study}/{approach}/sim.csv')
    
    #Rename the columns named 'end_timestamp' and 'start_timestamp' with 'time:timestamp' and 'start:timestamp'
    log_sim = log_sim.rename(columns={'end_timestamp': 'time:timestamp', 
                                      'start_timestamp': 'start:timestamp', 
                                      'task':'concept:name',
                                      'caseid': 'case:concept:name'}, errors='ignore')
    
    # Errors='ignore' has been set because the log from francesca meneghello et al has some columns already correctly

    print('imported log from Sota, approach:', approach, ',columns renamed')

parsing log, completed traces :: 100%|██████████| 45/45 [00:00<00:00, 399.81it/s]

imported log from our approach


In [34]:
# convert event log format lifecycles
if 'lifecycle:transition' in log_real.columns:
    log_real = convert_log(log_real)
    log_real.rename(columns={'START': 'start:timestamp', 'END': 'time:timestamp'}, errors='ignore', inplace=True)
if 'lifecycle:transition' in log_sim.columns:
    log_sim = convert_log(log_sim)

 11%|█         | 5/45 [00:00<00:00, 40.19it/s]

100%|██████████| 45/45 [00:01<00:00, 44.71it/s]


In [35]:
# Set event log column ID mapping
event_log_ids = EventLogIDs(
    case="case:concept:name",
    activity="concept:name",
    start_time="start:timestamp",
    end_time="time:timestamp"
)

In [36]:
log_real[event_log_ids.start_time] = pd.to_datetime(log_real[event_log_ids.start_time], utc=True)
log_real[event_log_ids.end_time] = pd.to_datetime(log_real[event_log_ids.end_time], utc=True)

log_sim[event_log_ids.start_time] = pd.to_datetime(log_sim[event_log_ids.start_time], utc=True)
log_sim[event_log_ids.end_time] = pd.to_datetime(log_sim[event_log_ids.end_time], utc=True)

In [37]:
log_sim.columns

Index(['case:concept:name', 'concept:name', 'org:resource', 'start:timestamp',
       'time:timestamp'],
      dtype='object')

In [38]:
#Initialize distances dictionary
distances = {}

### Control-flow Log Distance

<b>WARNING: It may take a long time</b>

In [39]:
# Call passing the event logs, and its column ID mappings
distance = control_flow_log_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
)

In [40]:
print('CF Log distance: ', distance)

# Fill the distances dictionary
distances['control_flow_log_distance'] = distance


CF Log distance:  0.4384692383323182


### N-Gram Distribution Distance

In [41]:
n_gram = 5

In [42]:
# Call passing the event logs, and its column ID mappings
distance = n_gram_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    n=n_gram
)

In [43]:
print('N-Gram distr. distance: ', distance)
# Fill the distances dictionary
distances['n_gram_distribution_distance'] = distance

N-Gram distr. distance:  0.48905109489051096


### Absolute Event Distribution Distance

In [44]:
# EMD of the (END) timestamps distribution where each bin represents a minute
distance = absolute_event_distribution_distance(
    log_real, event_log_ids,
    log_sim, event_log_ids,
    discretize_type=AbsoluteTimestampType.END,
    discretize_event=discretize_to_hour
)

In [45]:
print('Absolute Event Distribution Distance: ', distance)
# Fill the distances dictionary
distances['absolute_event_distribution_distance'] = distance

Absolute Event Distribution Distance:  1610.733345382252


### Case Arrival Distribution Distance

In [46]:
distance = case_arrival_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_event=discretize_to_hour  # Function to discretize each timestamp (default by hour)
)

In [47]:
print('Case Arrival distr distance: ', distance)
# Fill the distances dictionary
distances['case_arrival_distribution_distance'] = distance

Case Arrival distr distance:  1216.2888888888888


### Circadian Event Distribution Distance

In [48]:
distance = circadian_event_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_type=AbsoluteTimestampType.BOTH  # Consider both start/end timestamps of each activity instance
)

In [49]:
print('Circadian Event distr distance: ', distance)
# Fill the distances dictionary 
distances['circadian_event_distribution_distance'] = distance

Circadian Event distr distance:  2.7675003603184316


### Resource-Based Circadian Event Distribution Distance


In [50]:
distance = resource_based_circadian_event_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_type=AbsoluteTimestampType.BOTH  # Consider both start/end timestamps of each activity instance
)

In [51]:
print('Resource-Based Circadian Event distr distance: ', distance)
# Fill the distances dictionary 
distances['resource_based_circadian_event_distribution_distance'] = distance

Resource-Based Circadian Event distr distance:  7.9520930576496935


### Relative Event Distribution Distance

In [52]:
# Call passing the event logs, its column ID mappings, timestamp type, and discretize function
distance = relative_event_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
    discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
)

In [53]:
print('Relative Event distr distance: ', distance)
# Fill the distances dictionary
distances['relative_event_distribution_distance'] = distance

Relative Event distr distance:  94.31623983372491


### Work in Progress Distance

<b>WARNING: It may take a long time</b>

In [54]:
# Call passing the event logs, its column ID mappings, timestamp type, and discretize function
distance = work_in_progress_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    window_size=pd.Timedelta(hours=1)  # Bins of 1 hour
)

WARNING! The normalization of a Wasserstein Distance is sensitive to the range of the two samples, long samples may cause a higher reduction of the error.


In [55]:
print('Work in Progress distance: ', distance)
# Fill the distances dictionary
distances['work_in_progress_distance'] = 'NaN'

Work in Progress distance:  0.3376140634002855


### Cycle Time Distribution Distance

In [56]:
distance = cycle_time_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    bin_size=pd.Timedelta(hours=1)  # Bins of 1 minute
)

In [57]:
print('Cycle Time distr distance: ', distance)
# Fill the distances dictionary
distances['cycle_time_distribution_distance'] = distance

Cycle Time distr distance:  171.31111111111113


In [58]:
print(f'case study is {case_study} for approach {approach}, with our approach {our_approach} and Sota {Sota}')
distances

case study is Production for approach None, with our approach True and Sota False


{'control_flow_log_distance': 0.4384692383323182,
 'n_gram_distribution_distance': 0.48905109489051096,
 'absolute_event_distribution_distance': 1610.733345382252,
 'case_arrival_distribution_distance': 1216.2888888888888,
 'circadian_event_distribution_distance': 2.7675003603184316,
 'resource_based_circadian_event_distribution_distance': 7.9520930576496935,
 'relative_event_distribution_distance': 94.31623983372491,
 'work_in_progress_distance': 'NaN',
 'cycle_time_distribution_distance': 171.31111111111113}

## Purchasing Old

| Model         | Control Flow Log Distance | N-Gram Distribution Distance | Absolute Event Distribution Distance | Case Arrival Distribution Distance | Circadian Event Distribution Distance | Resource-Based Circadian Event Distribution Distance | Relative Event Distribution Distance | Work in Progress Distance | Cycle Time Distribution Distance |
|---------------|---------------------------|-------------------------------|------------------------------------|-----------------------------------|----------------------------------------|----------------------------------------------------|-----------------------------------|--------------------------|----------------------------------|
| LSTM          | 0.4864                    | 0.8282                        | 1300.409                           | 847.525                           | 2.5268                                     | 2.5268                                             | 826.602                           | NaN                      | 698.3427                         |
| Our Approach  | 0.3655                    | 0.2872                        | 963.281                            | 769.008                           | 0.7557                                     | 0.7557                                             | 536.862                           | NaN                      | 444.0492                         |
| RIMS          | 0.4794                    | 0.4953                        | 1079.587                           | 769.022                           | 0.9369                                     | NaN                                                | 688.982                           | NaN                      | 631.5424                         |
| SIMOD         | 0.5226                    | 0.6549                        | 787.306                            | 1265.784                          | 4.7083                                     | NaN                                                | 787.647                           | NaN                      | 588.9605                         |
| DSIM          | 0.1616                    | 0.3632                        | 1134.681                           | 774.761                           | 0.9966                                     | NaN                                                | 722.356                           | NaN                      | 596.9468                         |
| LSTM (GAN)    | 0.8572                    | 0.9908                        | 83375.183                          | 83832.262                         | 3.5611                                     | NaN                                                | 782.089                           | NaN                      | 638.1056                         |


## Production updated

| Model         | Control Flow Log Distance | N-Gram Distribution Distance | Absolute Event Distribution Distance | Case Arrival Distribution Distance | Circadian Event Distribution Distance | Resource-Based Circadian Event Distribution Distance | Relative Event Distribution Distance | Work in Progress Distance | Cycle Time Distribution Distance |
|---------------|---------------------------|-------------------------------|------------------------------------|-----------------------------------|----------------------------------------|----------------------------------------------------|-----------------------------------|--------------------------|----------------------------------|
| Our Approach  | 0.4485                    | 0.489                        | 1610.733                           | 1216.288                          | 2.6437                                     | 11.1975                                            | 94.316                           | NaN                      | 171.31                         |
| LSTM          | 0.8696                    | 1.0                           | 389.0209                           | 441.5556                          | 18.0232                                    | NaN                                                | 275.048                           | NaN                      | 360.5556                         |
| DSIM          | 0.7882                    | 0.9990                        | 378.0306                           | 443.1778                          | 2.5162                                     | NaN                                                | 227.643                           | NaN                      | 301.7556                         |
| LSTM (GAN)    | 0.9078                    | 1.0                           | 78504.7324                         | 78575.9556                        | 6.7231                                     | NaN                                                | 232.318                           | NaN                      | 313.1556                         |
| SIMOD         | 0.6588                    | 0.8939                        | 1962.0009                          | 1884.0000                         | 3.7217                                     | NaN                                                | 263.869                           | NaN                      | 348.3111                         |
| RIMS          | 0.7700                    | 0.9989                        | 371.8265                           | 446.3111                          | 2.5407                                     | NaN                                                | 216.239                           | NaN                      | 285.1333                         |
